# Wstęp

Celem tego projektu jest przeprowadzenie klasyfikacji na wybranej bazie danych z wykorzystaniem podstawowych klasyfikatorów:

- [_K najbliższych sąsiadów_](https://pl.wikipedia.org/wiki/K_najbli%C5%BCszych_s%C4%85siad%C3%B3w),
- [_naiwny klasyfikator bayesowski_](https://pl.wikipedia.org/wiki/Naiwny_klasyfikator_bayesowski),
- [_drzewo decyzyjne_](https://pl.wikipedia.org/wiki/Drzewo_decyzyjne),
- [_sieć neuronowa_](https://pl.wikipedia.org/wiki/Sie%C4%87_neuronowa).

Wybrana przeze mnie baza danych zawiera informacje o [diamentach](https://www.kaggle.com/datasets/shivam2503/diamonds).
Składa się ona z $53941$ wierszy i następujących $11$ kolumn:

1. **indeks**
2. **ilość karatów** (carat) - wartości od $0.2$ do $5.01$
3. **jakość cięcia** (quality of the cut) - wartości _Fair_, _Good_, _Very Good_, _Premium_, _Ideal_
4. **kolor** (color) - wartości od _J_ (najgorszy) do _D_ (najlepszy)
5. **klarowność** (clarity) - wartości _I1_ (najgorsza), _SI2_, _SI1_, _VS2_, _VS1_, _VVS2_, _VVS1_, _IF_ (najlepsza)
6. **dlugość** (x) - długość w milimetrach, wartości od $0$ do $10.74$
7. **szerokość** (y) - szerokość w milimetrach, wartości od $0$ do $58.9$
8. **głębokość** (z) - głębokość w milimetrach, wartości od $0$ do $31.8$
9. **łączny procent głębokości** (depth) - wyliczony ze wzoru $2z/(x + y)$, wartości od $43$ do $79$
10. **szerokość szczytu diamentu względem jego najszerszego punktu** (table) - wartości od $43$ do $95$
11. **cena** (price) - cena diamentu w dolarach, od $\$326$ do $\$18,823$

W mojej pracy będę starał się przewidzieć cenę diamentu na podstawie jego pozostałych właściwości.

# Przygotowanie danych

W celu przeprowadzenia klasyfikacji przygotuje $2$ bazy o różnym stopniu modyfikacji i przygotowania danych.
Wykorzystane do tego zostaną następujące biblioteki:

- [_Pandas_](https://pandas.pydata.org/)
- [_NumPy_](https://numpy.org/)
- [_scikit-learn_](https://scikit-learn.org/stable/)
- [_SciPy_](https://scipy.org/)

W poniższym fragmencie kodu zostają one zaimportowane, wczytany jest także plik [diamonds.csv](diamonds.csv) zawierający dane.
Poniżej jest także wyświetlona krótka charakterystyka danych:


In [129]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from scipy import stats

diamonds = pd.read_csv("diamonds.csv", index_col=0)

diamonds.describe()


,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


## Przygotowanie pierwszej bazy danych

Pierwsza baza danych zostanie mniej przetworzona niż druga i ulegnie jedynie podstawowym modyfikacjom.

### Zamiana wartości tekstowych na liczby

Pierwszym krokiem będzie zamiana wartości tekstowych na liczbowe, ponieważ wybrane algorytmy klasyfikujące nie są w stanie działać na danych tekstowych.
W wybranej bazie znajdują się $3$ kolumny z wartościami tekstowymi: _jakość cięcia_ (cut), _kolor_ (color) oraz _klarowność_ (clarity).
Wartości zostaną zmienione w następujący sposób:

<table><thead><tr><th colspan="2">Jakość cięcia</th><th colspan="2">Kolor<br></th><th colspan="2">Klarowność</th></tr></thead><tbody><tr><td>Przed zmianą</td><td>Po zmianie</td><td>Przed zmianą</td><td>Po zmianie<br></td><td>Przed zmianą</td><td>Po zmianie</td></tr><tr><td>Fair<br></td><td>0</td><td>D</td><td>0</td><td>I1</td><td>0</td></tr><tr><td>Good</td><td>1</td><td>E</td><td>1</td><td>SI2</td><td>1</td></tr><tr><td>Very Good</td><td>2</td><td>F</td><td>2</td><td>SI1</td><td>2</td></tr><tr><td>Premium</td><td>3</td><td>G</td><td>3</td><td>VS2</td><td>3</td></tr><tr><td>Ideal</td><td>4</td><td>H</td><td>4</td><td>VS1</td><td>4</td></tr><tr><td colspan="2" rowspan="3"></td><td>I</td><td>5</td><td>VVS2</td><td>5</td></tr><tr><td>J</td><td>6</td><td>VVS1</td><td>6</td></tr><tr><td colspan="2"></td><td>IF</td><td>7</td></tr></tbody></table>

Do przetworzenia $2$ pierwszych kolumn można wykorzystać obiekt klasy _LabelEncoder_ dostępnej w bibliotece _sklearn_, natomiast do modyfikacji klarowności posłuży specjalnie zdefiniowana funkcja:


In [130]:
def change_clarity(value):
    res = 0
    match value:
        case "I1": res = 0
        case "SI2": res = 1
        case "SI1": res = 2
        case "VS2": res = 3
        case "VS1": res = 4
        case "VVS2": res = 5
        case "VVS1": res = 6
        case "IF": res = 7
        case _: res = value

    return res


LE = LabelEncoder()

first_database = diamonds.copy()

first_database.sort_values(by="cut", inplace=True)
first_database["cut"] = LE.fit_transform(first_database["cut"])

first_database.sort_values(by="color", inplace=True)
first_database["color"] = LE.fit_transform(first_database["color"])

first_database["clarity"] = first_database["clarity"].apply(change_clarity)


Po zmianie dane prezentują się w następujący sposób:


In [131]:
first_database.describe()


,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.553003,2.594197,3.051020,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.027708,1.701105,1.647136,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,3.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,4.000000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,4.000000,6.000000,7.000000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


### Usunięcie niepoprawnych danych

Drugim i ostatnim krokiem w przygotowaniu pierwszej bazy jest usunięcie niepoprawnych danych.
W tym przypadku są to wiersze zawierające wartości $x$, $y$ lub $z$ równe $0$ - te dane opisują diamenty, które nie mają zdefiniowanego przynajmniej jednego ze swoich wymiarów, wobec tego nie mogą występować w przyrodzie.


In [132]:
first_database = first_database[first_database["x"] != 0]
first_database = first_database[first_database["y"] != 0]
first_database = first_database[first_database["z"] != 0]


Usunęliśmy w ten sposób $20$ niepoprawnych wierszy.
Poniżej znajduje się przygotowana pierwsza baza danych:


In [133]:
first_database.describe()


,carat,cut,color,clarity,depth,table,price,x,y,z
count,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000,53920.000000
mean,0.797698,2.553079,2.594028,3.051502,61.749514,57.456834,3930.993231,5.731627,5.734887,3.540046
std,0.473795,1.027705,1.701272,1.647005,1.432331,2.234064,3987.280446,1.119423,1.140126,0.702530
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,326.000000,3.730000,3.680000,1.070000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,949.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,3.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,4.000000,62.500000,59.000000,5323.250000,6.540000,6.540000,4.040000
max,5.010000,4.000000,6.000000,7.000000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


## Przygotowanie drugiej bazy danych

Pierwsze dwa etapy przygotowania danych są identycznie dla obu baz, wobec tego można skopiować zawartość pierwszej z nich w celu dalszej modyfikacji:


In [134]:
second_database = first_database.copy()


### Usunięcie danych odstających

Etap ten polega na usunięciu danych, które odstają od średnich wartości.
W tym celu wykorzystamy tzw. [z-score](https://en.wikipedia.org/wiki/Standard_score) (standardowy wynik), czyli stopień odchylenia wartości od średniej.
Usunięte zostaną dane, dla których $|z-score| < 3$.


In [135]:
z_score = np.abs(stats.zscore(second_database))

second_database = second_database[(z_score < 3).all(axis=1)]


W ten sposób usunięte zostało $2334$ skrajnych wierszy. Po tym etapie druga baza danych wygląda następująco:


In [136]:
second_database.describe()


,carat,cut,color,clarity,depth,table,price,x,y,z
count,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000
mean,0.759831,2.589772,2.566316,3.087039,61.752815,57.368972,3585.077230,5.657664,5.661082,3.494530
std,0.424843,0.990101,1.694661,1.642576,1.269269,2.100011,3435.297118,1.057236,1.050201,0.652934
min,0.200000,0.000000,0.000000,0.000000,57.500000,51.000000,326.000000,3.730000,3.680000,1.530000
25%,0.390000,2.000000,1.000000,2.000000,61.100000,56.000000,926.000000,4.690000,4.700000,2.890000
50%,0.700000,2.000000,3.000000,3.000000,61.800000,57.000000,2303.000000,5.660000,5.660000,3.490000
75%,1.020000,3.000000,4.000000,4.000000,62.500000,59.000000,5047.000000,6.490000,6.490000,4.010000
max,2.210000,4.000000,6.000000,7.000000,66.000000,64.000000,15889.000000,8.600000,8.550000,5.300000


### Normalizacja danych

Ostatnim krokiem przygotowania drugiej bazy jest znormalizowanie danych i przeskalowanie ich tak, aby wartości mieściły się w zakresie $[0, 1]$.
Kolumny, które nie zostaną przeskalowane to: _jakość cięcia_ (cut), _kolor_ (color), _klarowność_ (clarity) oraz _cena_ (price).


In [137]:
columns_to_normalize = ["carat", "depth", "table", "x", "y", "z"]
scaler = MinMaxScaler()

values = second_database[columns_to_normalize].values
scaled_values = scaler.fit_transform(values)

second_database[columns_to_normalize] = pd.DataFrame(
    scaled_values, columns=columns_to_normalize, index=second_database.index)


Po ostatecznym zmodyfikowaniu danych druga baza ma poniższą postać:


In [138]:
second_database.describe()


,carat,cut,color,clarity,depth,table,price,x,y,z
count,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000,51586.000000
mean,0.278523,2.589772,2.566316,3.087039,0.500331,0.489921,3585.077230,0.395824,0.406793,0.521095
std,0.211364,0.990101,1.694661,1.642576,0.149326,0.161539,3435.297118,0.217092,0.215647,0.173192
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,326.000000,0.000000,0.000000,0.000000
25%,0.094527,2.000000,1.000000,2.000000,0.423529,0.384615,926.000000,0.197125,0.209446,0.360743
50%,0.248756,2.000000,3.000000,3.000000,0.505882,0.461538,2303.000000,0.396304,0.406571,0.519894
75%,0.407960,3.000000,4.000000,4.000000,0.588235,0.615385,5047.000000,0.566735,0.577002,0.657825
max,1.000000,4.000000,6.000000,7.000000,1.000000,1.000000,15889.000000,1.000000,1.000000,1.000000


# Przeprowadzenie klasyfikacji

Następnym etapem po przygotowaniu danych będzie przeprowadzenie klasyfikacji na obu bazach danych z wykorzystaniem różnych algorytmów.
Przed przystąpieniem do tego etapu należy podzielić bazy na zbiory testowe oraz treningowe.
W tym celu wykorzystam


# Podsumowanie


# Bibliografia

- [dokumentacja Pandas](https://pandas.pydata.org/)
- [dokumentacja NumPy](https://numpy.org/)
- [dokumentacja SciPy](https://scipy.org/)
- [dokumentacja scikit-learn](https://scikit-learn.org/stable/)
